In [1]:
import xpress as xp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyreadr import read_r
from scipy.interpolate import interp1d
from param_watervalues import Reservoir
from time import time

In [2]:
%load_ext line_profiler

In [3]:
%matplotlib widget

In [4]:
xp.controls.outputlog = -1

In [5]:
study_path = "D:/Users/gerbauxjul/Documents/6-Etudes Antares/OneNodeBase_v90_overflow"
output_path = study_path+"/user"

In [6]:
def retrieve_problem(year,week,output_path,itr=1):
    model = xp.problem()
    model.read(output_path+f"/problem-{year}-{week}--optim-nb-{itr}.mps")
    return(model)

In [7]:
s = 0
k = 0

In [8]:
m = retrieve_problem(k+1,s+1,output_path,itr=1)

In [9]:
vars = m.getVariable()

In [10]:
df_vars = pd.DataFrame(vars, columns=["names"])
df_vars["split"] = df_vars["names"].apply(lambda x:x.name.strip().split("::"))
df_vars["name_var"] = df_vars["split"].apply(lambda x:x[0])
df_vars["antares_object"] = df_vars["split"].apply(lambda x:x[1].split("<")[0])
df_vars["name_antares_object"] = df_vars["split"].apply(lambda x:x[1].split("<")[1].split(">")[0])
df_vars["subobject"] = df_vars["split"].apply(lambda x:x[2] if len(x)>=4 else "None")
df_vars["time"] = df_vars["split"].apply(lambda x:int(x[-1].split("<")[1].split(">")[0]))
coltype = []
m.getcoltype(coltype, 0, m.attributes.cols - 1)
df_vars['coltype'] = coltype
bounds = []
m.getlb(bounds, 0, m.attributes.cols - 1)
df_vars['lb'] = bounds
bounds = []
m.getub(bounds, 0, m.attributes.cols - 1)
df_vars['ub'] = bounds
objcoef = []
m.getobj(objcoef,0,m.attributes.cols-1)
df_vars= df_vars.assign(coeff=[objcoef[i] for i in range(m.attributes.cols)])
df_vars = df_vars.assign(sol=[m.getSolution(i) for i in range(m.attributes.cols)])
df_vars = df_vars.assign(obj = lambda x: np.round(df_vars['sol'])*np.round(df_vars['coeff']))

In [11]:
df_vars

,names,split,name_var,antares_object,name_antares_object,subobject,time,coltype,lb,ub,coeff,sol,obj
0,DispatchableProduction::area<area>::ThermalClu...,"[DispatchableProduction, area<area>, ThermalCl...",DispatchableProduction,area,area,ThermalCluster<Th1>,0,C,0.0,4.250000e+04,0.099456,0.0,0.0
1,DispatchableProduction::area<area>::ThermalClu...,"[DispatchableProduction, area<area>, ThermalCl...",DispatchableProduction,area,area,ThermalCluster<Th2>,0,C,0.0,1.250000e+04,0.199493,0.0,0.0
2,DispatchableProduction::area<area>::ThermalClu...,"[DispatchableProduction, area<area>, ThermalCl...",DispatchableProduction,area,area,ThermalCluster<Th3>,0,C,0.0,7.500000e+03,0.300570,0.0,0.0
3,PositiveUnsuppliedEnergy::area<area>::hour<0> ...,"[PositiveUnsuppliedEnergy, area<area>, hour<0>]",PositiveUnsuppliedEnergy,area,area,None,0,C,0.0,3.704000e+04,2.999412,0.0,0.0
4,NegativeUnsuppliedEnergy::area<area>::hour<0> ...,"[NegativeUnsuppliedEnergy, area<area>, hour<0>]",NegativeUnsuppliedEnergy,area,area,None,0,C,0.0,1.000000e+20,-0.000588,0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,NegativeUnsuppliedEnergy::area<area>::hour<167...,"[NegativeUnsuppliedEnergy, area<area>, hour<167>]",NegativeUnsuppliedEnergy,area,area,None,167,C,0.0,1.000000e+20,-0.000588,0.0,-0.0
1508,HydProd::area<area>::hour<167> ...,"[HydProd, area<area>, hour<167>]",HydProd,area,area,None,167,C,0.0,5.000000e+04,-0.000879,0.0,-0.0
1509,Pumping::area<area>::hour<167> ...,"[Pumping, area<area>, hour<167>]",Pumping,area,area,None,167,C,0.0,5.000000e+04,0.001757,0.0,0.0
1510,HydroLevel::area<area>::hour<167> ...,"[HydroLevel, area<area>, hour<167>]",HydroLevel,area,area,None,167,C,0.0,1.000000e+07,0.000000,0.0,0.0


In [12]:
np.unique(df_vars["name_antares_object"])

array(['area'], dtype=object)

In [13]:
df_vars[(df_vars["name_var"]=='HydProd')].groupby("name_antares_object")["sol"].sum()

name_antares_object
area    0.0
Name: sol, dtype: float64

In [14]:
cons = m.getConstraint()

In [15]:
len(cons)

508

In [16]:
df_cons = pd.DataFrame(cons, columns=["names"])
df_cons["split"] = df_cons["names"].apply(lambda x:x.name.strip().split("::"))
df_cons["name_cons"] = df_cons["split"].apply(lambda x:x[0])
df_cons["antares_object"] = df_cons["split"].apply(lambda x:x[1].split("<")[0])
df_cons["name_antares_object"] = df_cons["split"].apply(lambda x:x[1].split("<")[1].split(">")[0] if len(x[1].split("<"))>=2 else "None")
df_cons["time"] = df_cons["split"].apply(lambda x:int(x[-1].split("<")[1].split(">")[0]))
rhs = []
m.getrhs(rhs, 0, m.attributes.rows - 1)
df_cons["rhs"] = rhs

In [17]:
df_cons

,names,split,name_cons,antares_object,name_antares_object,time,rhs
0,AreaBalance::area<area>::hour<0> ...,"[AreaBalance, area<area>, hour<0>]",AreaBalance,area,area,0,-37040.000000
1,FictiveLoads::area<area>::hour<0> ...,"[FictiveLoads, area<area>, hour<0>]",FictiveLoads,area,area,0,0.000000
2,AreaBalance::area<area>::hour<1> ...,"[AreaBalance, area<area>, hour<1>]",AreaBalance,area,area,1,-42837.000000
3,FictiveLoads::area<area>::hour<1> ...,"[FictiveLoads, area<area>, hour<1>]",FictiveLoads,area,area,1,0.000000
4,AreaBalance::area<area>::hour<2> ...,"[AreaBalance, area<area>, hour<2>]",AreaBalance,area,area,2,-49172.000000
...,...,...,...,...,...,...,...
503,AreaHydroLevel::area<area>::hour<163> ...,"[AreaHydroLevel, area<area>, hour<163>]",AreaHydroLevel,area,area,163,190.541667
504,AreaHydroLevel::area<area>::hour<164> ...,"[AreaHydroLevel, area<area>, hour<164>]",AreaHydroLevel,area,area,164,190.541667
505,AreaHydroLevel::area<area>::hour<165> ...,"[AreaHydroLevel, area<area>, hour<165>]",AreaHydroLevel,area,area,165,190.541667
506,AreaHydroLevel::area<area>::hour<166> ...,"[AreaHydroLevel, area<area>, hour<166>]",AreaHydroLevel,area,area,166,190.541667


In [18]:
np.unique(df_cons["name_cons"])

array(['AreaBalance', 'AreaHydroLevel', 'FictiveLoads', 'HydroPower',
       'MaxHydroPower', 'MaxPumping', 'MinHydroPower'], dtype=object)

In [19]:
df_cons[df_cons["time"]==0]

,names,split,name_cons,antares_object,name_antares_object,time,rhs
0,AreaBalance::area<area>::hour<0> ...,"[AreaBalance, area<area>, hour<0>]",AreaBalance,area,area,0,-3.704000e+04
1,FictiveLoads::area<area>::hour<0> ...,"[FictiveLoads, area<area>, hour<0>]",FictiveLoads,area,area,0,0.000000e+00
336,HydroPower::area<area>::week<0> ...,"[HydroPower, area<area>, week<0>]",HydroPower,area,area,0,1.934044e+06
337,MinHydroPower::area<area>::week<0> ...,"[MinHydroPower, area<area>, week<0>]",MinHydroPower,area,area,0,0.000000e+00
338,MaxHydroPower::area<area>::week<0> ...,"[MaxHydroPower, area<area>, week<0>]",MaxHydroPower,area,area,0,8.400000e+06
339,MaxPumping::area<area>::week<0> ...,"[MaxPumping, area<area>, week<0>]",MaxPumping,area,area,0,8.400000e+06
340,AreaHydroLevel::area<area>::hour<0> ...,"[AreaHydroLevel, area<area>, hour<0>]",AreaHydroLevel,area,area,0,4.866667e+06


In [20]:
df_vars[df_vars["time"]==0]

,names,split,name_var,antares_object,name_antares_object,subobject,time,coltype,lb,ub,coeff,sol,obj
0,DispatchableProduction::area<area>::ThermalClu...,"[DispatchableProduction, area<area>, ThermalCl...",DispatchableProduction,area,area,ThermalCluster<Th1>,0,C,0.0,4.250000e+04,0.099456,0.0,0.0
1,DispatchableProduction::area<area>::ThermalClu...,"[DispatchableProduction, area<area>, ThermalCl...",DispatchableProduction,area,area,ThermalCluster<Th2>,0,C,0.0,1.250000e+04,0.199493,0.0,0.0
2,DispatchableProduction::area<area>::ThermalClu...,"[DispatchableProduction, area<area>, ThermalCl...",DispatchableProduction,area,area,ThermalCluster<Th3>,0,C,0.0,7.500000e+03,0.300570,0.0,0.0
3,PositiveUnsuppliedEnergy::area<area>::hour<0> ...,"[PositiveUnsuppliedEnergy, area<area>, hour<0>]",PositiveUnsuppliedEnergy,area,area,None,0,C,0.0,3.704000e+04,2.999412,0.0,0.0
4,NegativeUnsuppliedEnergy::area<area>::hour<0> ...,"[NegativeUnsuppliedEnergy, area<area>, hour<0>]",NegativeUnsuppliedEnergy,area,area,None,0,C,0.0,1.000000e+20,-0.000588,0.0,-0.0
5,HydProd::area<area>::hour<0> ...,"[HydProd, area<area>, hour<0>]",HydProd,area,area,None,0,C,0.0,5.000000e+04,-0.000945,0.0,-0.0
6,Pumping::area<area>::hour<0> ...,"[Pumping, area<area>, hour<0>]",Pumping,area,area,None,0,C,0.0,5.000000e+04,0.001890,0.0,0.0
7,HydroLevel::area<area>::hour<0> ...,"[HydroLevel, area<area>, hour<0>]",HydroLevel,area,area,None,0,C,0.0,1.000000e+07,0.000000,0.0,0.0
8,Overflow::area<area>::hour<0> ...,"[Overflow, area<area>, hour<0>]",Overflow,area,area,None,0,C,0.0,4.166667e+05,-0.000588,0.0,-0.0


In [21]:
10000000/24

416666.6666666667

In [22]:
start = []
colind = []
colcoef = []

In [23]:
m.getrows(start=start,colind=colind,colcoef=colcoef,maxcoefs=600,first=336,last=336)

In [24]:
start

[0, 336]

In [25]:
colind

[HydProd::area<area>::hour<0>                                            ,
 Pumping::area<area>::hour<0>                                            ,
 HydProd::area<area>::hour<1>                                            ,
 Pumping::area<area>::hour<1>                                            ,
 HydProd::area<area>::hour<2>                                            ,
 Pumping::area<area>::hour<2>                                            ,
 HydProd::area<area>::hour<3>                                            ,
 Pumping::area<area>::hour<3>                                            ,
 HydProd::area<area>::hour<4>                                            ,
 Pumping::area<area>::hour<4>                                            ,
 HydProd::area<area>::hour<5>                                            ,
 Pumping::area<area>::hour<5>                                            ,
 HydProd::area<area>::hour<6>                                            ,
 Pumping::area<area>::hou

In [26]:
colcoef

[1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,

In [27]:
m.solve()

In [28]:
m.getProbStatusString()

'lp_optimal'